In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
from machinedesign.autoencoder.interface import load
import keras.backend as K
from keras.models import Model
import numpy as np
from skimage.io import imsave
from sklearn.neighbors import NearestNeighbors
from machinedesign.viz import grid_of_images_default


In [152]:
data = np.load('../results/jobs/0debb69cc1446f1d7a5a96f811862415/gen/generated.npz')
X = data['generated']
clf = load('../../discr/digits')
clf_enc = Model(inputs=clf.layers[0].input, outputs=clf.get_layer('p_re_lu_4').output)
model = load('../results/jobs/0debb69cc1446f1d7a5a96f811862415')
inp = model.layers[0].input
hid_layer = model.get_layer('winner_take_all_spatial_1')
hid = hid_layer.output
out = model.layers[-1].output
enc = K.function([inp], hid)
dec = K.function([hid], out)

In [153]:
def F(x):
    h = enc([x])
    h = h.max(axis=(2, 3))
    #h = (h>0).astype(int)
    h = h.reshape((h.shape[0], -1))
    return h

In [154]:
digits = np.load('../../data/digits.npz')['X'] / 255.0
htrue = F(digits)

In [186]:
mu = np.median(htrue, axis=0, keepdims=True)
htrue_binary = htrue > mu
htrue_binary = htrue_binary.astype(int)

In [188]:
knn = NearestNeighbors(n_neighbors=5, metric='hamming')
knn.fit(htrue)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='hamming',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [189]:
x = X[0:10]
hfake = F(x)
hfake_binary = hfake > mu
hfake_binary = hfake_binary.astype(int)
print(hfake_binary)

[[1 1 1 1 1 1 1 1]
 [1 1 1 0 0 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 0 1 1 1 1]
 [1 1 1 1 1 1 1 1]]


In [167]:
dist, ind = knn.kneighbors(hfake, return_distance=True)
dist *= hfake.shape[1]
#dist = dist.astype(int)

In [168]:
xtrue = digits[ind[:, 0]]
xfake = x 
xall = np.concatenate((xfake, xtrue), axis=0)
im = grid_of_images_default(xall, shape=(2, len(xtrue)))
imsave('out.png', im)
print(xall.shape)

(200, 1, 28, 28)


/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [185]:
hh = enc([x])
hh = hh > mu.reshape((mu.shape[0], mu.shape[1],1,1))
hh = hh.astype('float32')
hh *= 30
print(hh.shape)
xx = dec([hh])
im = grid_of_images_default(xx)
imsave('out.png', im)

(10, 8, 16, 16)


/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
